## Experiment Goal

The goal of this experiment is to research what a complete observation space would look like.

In [2]:
import gymnasium as gym
from sklearn.preprocessing import LabelEncoder

In [3]:
# data imports
import pandas as pd
import os
import poke_battle_sim as pb

# Get the package directory
package_dir = str(os.sep).join(str(pb.poke_sim.__file__).split(os.sep)[0:-1])
data_dir = os.path.join(package_dir, 'data')

# Load dataframes
abilities = pd.read_csv(os.path.join(data_dir, 'abilities.csv'))
items_gen4 = pd.read_csv(os.path.join(data_dir, 'items_gen4.csv'))
move_list = pd.read_csv(os.path.join(data_dir, 'move_list.csv'))
natures = pd.read_csv(os.path.join(data_dir, 'natures.csv'))
pokemon_stats = pd.read_csv(os.path.join(data_dir, 'pokemon_stats.csv'))
# pokemon_stats.set_index('ndex', inplace=True)
type_effectiveness = pd.read_csv(os.path.join(data_dir, 'type_effectiveness.csv'))

In [4]:
# data helper methods
import random

def get_random_nature():
    return random.choice(natures.values)

def get_stats_by_id(pokedex_id: int):
    if pokedex_id < min(pokemon_stats['ndex']) or pokedex_id > max(pokemon_stats['ndex']):
        raise ValueError(f'{pokedex_id} is not a valid pokedex id')
    
    return pb.PokeSim._pokemon_stats[pokedex_id - 1][4:10]

def get_stats_by_name(name: str):
    if name not in pokemon_stats['name'].values:
        raise ValueError(f'{name} is not a valid pokemon name')
    
    search_results = [ i for i in pb.PokeSim._pokemon_stats if i[1] == name ] # TODO make this search more time efficient
    if len(search_results) != 1:
        raise ValueError(f'Invalid search results: expected 1, got {len(search_results)} while searching for {name}')

    return search_results[0][4:10]

In [5]:
# encoding/decoding methods
gender_encoder = LabelEncoder()
gender_encoder.fit(pb.conf.global_settings.POSSIBLE_GENDERS)

def get_random_gender_mf():
    return gender_encoder.transform(random.choice(['male', 'female']))

def get_gender_encoding(gender: str):
    return gender_encoder.transform([gender])[0]

def get_gender_decoding(gender: int):
    return gender_encoder.inverse_transform([gender])[0]

## Informally Defined Observation Space

The observation space should include the following:
- The agents party
- That NPC's party
- Lingering effects on the battlefield, like:
  - Weather
  - Stealth rocks and other entry hazards
  - Reflect and Light screen effects
  - etc
- Volatile status applied to a Pokémon out in battle
  - Stat changes from buffs and debuffs (like attack up, defense down, etc)
  - Effects like confusion, leech seed, flinching, etc
  - Trapping moves like whirlpool, fire spin, etc

A party consists of one to six Pokémon in the form of a dictionary, where the keys are the position of a Pokémon within the party (so key = 0 is the Pokémon in front of the party, key one the next Pokémon in the party etc).

A Pokémon is a tuple with:
- Stat totals (computed stats based on EV’s, IV’s, Base stats, Level and nature)
- Types (one or two types)
- Its ability
- Available moves
- Non-volatile status effects (like sleep, poison, etc)
- Held item
- Its weight (used for moves like low kick and grass knot)
- Current friendship value (used for moves like return and frustration)
- Its gender (used for moves like attract)

Some more notes on the observation space:
- For the agent's party, all this information is known beforehand. 
- For the NPC's party, the agent will have to learn how to gather this information throughout every episode.

To get these observations, we will first define it and then see how we can get it from the simulator.

## Finding min and max for every stat

| Stat | Min | Max | Type     |
|------|-----|-----|----------|
| HP   | 0   | 714 | Discrete |
| Atk  | 4   | 471 | Discrete |
| Def  | 4   | 614 | Discrete |
| SpA  | 4   | 447 | Discrete |
| SpD  | 4   | 614 | Discrete |
| Spe  | 4   | 460 | Discrete |

---

One might be tempted to just use the described values from the dataframe:

In [6]:
stat_columns = ['hp', 'attack', 'defense', 'sp. atk', 'sp. def', 'speed']

In [7]:
pokemon_stats[stat_columns].describe()

,hp,attack,defense,sp. atk,sp. def,speed
count,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000
mean,67.730223,73.496957,70.109533,67.981744,69.158215,65.440162
std,27.580375,29.168464,30.703012,28.515038,27.884112,27.223685
min,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,50.000000,50.000000,50.000000,45.000000,50.000000,45.000000
50%,65.000000,72.000000,65.000000,65.000000,65.000000,65.000000
75%,80.000000,90.000000,85.000000,90.000000,85.000000,85.000000
max,255.000000,165.000000,230.000000,154.000000,230.000000,160.000000


However, stats are actually not bounded by these values, as the came calulates the stats based on the following formula:

$$
hp(p) = \left[ \frac{2 \times \text{base} + \text{iv} + (\frac{\text{ev}}{4}) \times \text{level}}{100} \right] + level + 10
$$
All other stats: $$ f(p) = \left[ \left( \left[ \frac{2 \times \text{base } + \text{ iv} + (\frac{\text{ ev}}{4}) \times \text{level}}{100} \right] + 5 \right) \times \text{nature} \right] $$

We could use the theoratical maximum integer value for the stats, but that would be a bit of a waste of space as no pokemon will ever reach those values. Instead we will use the data from the dataframe to get the maximum and minimum (practically) possible values for each stat.

In [8]:
def calc_hp(base: int, ev = 252, iv = 31, lvl = 100):
    # Taken from %ENV-DIR%/poke_battle_sim/poke_sim/pokemon.py::Pokemon::calculate_stats_actual
    # stats_actual.append(
    #     ((2 * self.base[0] + self.ivs[0] + self.evs[0] // 4) * self.level) // 100 + 10
    # )
    return int(((2 * base + iv + ev // 4) * lvl) // 100 + lvl + 10)

def calc_stat(base: int, ev = 252, iv = 31, lvl = 100, nature = 1.1): 
    # Taken from %ENV-DIR%/poke_battle_sim/poke_sim/pokemon.py::Pokemon::calculate_stats_actual
    # stats_actual.append(
    #     (
    #         ((2 * self.base[s] + self.ivs[s] + self.evs[s] // 4) * self.level)
    #         // 100
    #         + 5
    #     )
    #     * nature_stat_changes[s]
    # )
    return int((((2 * base + iv + ev // 4) * lvl) // 100 + 5) * nature)

In [9]:
# quick sanity check
turtwig = pokemon_stats[pokemon_stats['name'] == 'turtwig'][stat_columns]
turtwig['hp'] = calc_hp(turtwig['hp'].values[0])
turtwig['attack'] = calc_stat(turtwig['attack'].values[0])
turtwig['defense'] = calc_stat(turtwig['defense'].values[0])
turtwig['sp. atk'] = calc_stat(turtwig['sp. atk'].values[0])
turtwig['sp. def'] = calc_stat(turtwig['sp. def'].values[0])
turtwig['speed'] = calc_stat(turtwig['speed'].values[0])
all(turtwig.values[0] == [314, 258, 249, 207, 229, 177])

True

In [10]:
max_computer_stats = pokemon_stats[stat_columns].copy()
max_computer_stats['hp'] = max_computer_stats['hp'].apply(lambda x: calc_hp(x))
max_computer_stats['attack'] = max_computer_stats['attack'].apply(lambda x: calc_stat(x))
max_computer_stats['defense'] = max_computer_stats['defense'].apply(lambda x: calc_stat(x))
max_computer_stats['sp. atk'] = max_computer_stats['sp. atk'].apply(lambda x: calc_stat(x))
max_computer_stats['sp. def'] = max_computer_stats['sp. def'].apply(lambda x: calc_stat(x))
max_computer_stats['speed'] = max_computer_stats['speed'].apply(lambda x: calc_stat(x))
max_computer_stats.describe()

,hp,attack,defense,sp. atk,sp. def,speed
count,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000
mean,339.460446,269.787018,262.330629,257.634888,260.235294,252.060852
std,55.160750,64.156830,67.531220,62.732855,61.338006,59.882815
min,206.000000,119.000000,119.000000,130.000000,152.000000,119.000000
25%,304.000000,218.000000,218.000000,207.000000,218.000000,207.000000
50%,334.000000,267.000000,251.000000,251.000000,251.000000,251.000000
75%,364.000000,306.000000,295.000000,306.000000,295.000000,295.000000
max,714.000000,471.000000,614.000000,447.000000,614.000000,460.000000


In [11]:
max_stats_row = max_computer_stats.loc[max_computer_stats.idxmax()]
pokemon_stats.loc[max_stats_row.index][['name']]

,name
241,blissey
408,rampardos
212,shuckle
149,mewtwo
212,shuckle
290,ninjask


The output above is corret: I know from experience that these are the pokemon with the highest stats in the game for each stat. Now for the minimum values... We can skip HP, since that has the lowst minimum of 0. 

In [12]:
min_computer_stats = pokemon_stats[['attack', 'defense', 'sp. atk', 'sp. def', 'speed']].copy()
min_computer_stats['attack'] = min_computer_stats['attack'].apply(lambda x: calc_stat(x, ev=0, iv=0, lvl=1, nature=0.9))
min_computer_stats['defense'] = min_computer_stats['defense'].apply(lambda x: calc_stat(x, ev=0, iv=0, lvl=1, nature=0.9))
min_computer_stats['sp. atk'] = min_computer_stats['sp. atk'].apply(lambda x: calc_stat(x, ev=0, iv=0, lvl=1, nature=0.9))
min_computer_stats['sp. def'] = min_computer_stats['sp. def'].apply(lambda x: calc_stat(x, ev=0, iv=0, lvl=1, nature=0.9))
min_computer_stats['speed'] = min_computer_stats['speed'].apply(lambda x: calc_stat(x, ev=0, iv=0, lvl=1, nature=0.9))
min_computer_stats.describe()

,attack,defense,sp. atk,sp. def,speed
count,493.000000,493.000000,493.000000,493.000000,493.000000
mean,5.004057,4.945233,4.898580,4.941176,4.843813
std,0.668684,0.706418,0.685104,0.655332,0.635939
min,4.000000,4.000000,4.000000,4.000000,4.000000
25%,5.000000,5.000000,4.000000,5.000000,4.000000
50%,5.000000,5.000000,5.000000,5.000000,5.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000
max,7.000000,8.000000,7.000000,8.000000,7.000000


In [13]:
min_stats_row = min_computer_stats.loc[min_computer_stats.idxmin()]
pokemon_stats.loc[min_stats_row.index][['name']]

,name
0,bulbasaur
0,bulbasaur
9,caterpie
9,caterpie
0,bulbasaur


This did not seem right at first, because I expected Kricketune to have the lowest defensive stats in the game. But after manual verification via various other calculators, I can confirm that these are the correct values.

## Label Encoding Types

| Type | Encoding |
|------|----------|
| bug | 0 |
| dark | 1 |
| dragon | 2 |
| electric | 3 |
| fighting | 4 |
| fire | 5 |
| flying | 6 |
| ghost | 7 |
| grass | 8 |
| ground | 9 |
| ice | 10 |
| normal | 11 |
| poison | 12 |
| psychic | 13 |
| rock | 14 |
| steel | 15 |
| water | 16 |
| nan | 17 |

These are luckily alot more easy to get. Since types are essentially a finite amount of strings (17 to be exact), we can simply label encode them. For future refrence and consitency sake I put the encodings in a markdown table above, just in case the `LabelEncoder` from sklearn gets updated or something.

In [14]:
type_encoder = LabelEncoder()
type_encoder.fit(pokemon_stats[[ 'type 1', 'type 2' ]].values.flatten())

def get_type_encoding(type_name: str):
    return type_encoder.transform([type_name])[0]

def get_type_decoding(type_id: int):
    return type_encoder.inverse_transform([type_id])[0]

# for c in type_encoder.classes_:
#     print(f'{c} -> {get_type_encoding(c)} -> {get_type_decoding(get_type_encoding(c))}')

## Ailities

... TODO FIX THIS SHIT

## Moves

| Column        | Min | Max | Type     |
|---------------|-----|-----|----------|
| id            | 1   | 467 | Discrete |
| type_id       | 1   | 17  | Discrete |
| power         | 10  | 250 | Discrete |
| pp            | 1   | 40  | Discrete |
| accuracy      | -2  | 100 | Discrete |
| priority      | -7  | 5   | Discrete |
| move_class    | 0   | 2   | Discrete |
| effect_id     | 0   | 219 | Discrete |
| effect_chance | 10  | 100 | Discrete |

The empty move will be defined as followed:

| Column          | Value                                          |
|-----------------|------------------------------------------------|
| `id`            | `starter_move_list['id'].min() - 1`            |
| `type_id`       | 17 (the `np.nan` encoded value)                |
| `power`         | `starter_move_list['power'].min() - 1`         |
| `pp`            | `starter_move_list['pp'].min() - 1`            |
| `accuracy`      | `starter_move_list['accuracy'].min() - 1`      |
| `priority`      | `starter_move_list['priority'].min() - 1`      |
| `target_id`     | `starter_move_list['target_id'].min() - 1`     |
| `move_class`    |`starter_move_list['move_class'].min() - 1`     |
| `effect_id`     |`starter_move_list['effect_id'].min() - 1`      |
| `effect_chance` | `starter_move_list['effect_chance'].min() - 1` |
| `effect_amt`    | `starter_move_list['effect_amt'].min() - 1`    |
| `effect_stat`   | `starter_move_list['effect_stat'].min() - 1`   |

This results in the following tuple:
> $\lambda = (0, 17, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1)$

A quick glance at the table shows me we can use:
- `id`, which removes the need for label encoding names
- `type_id`, which we will need to label encode but thats easy
- `power` and `pp` we can use as is
  - For any `power == np.nan` values we will use 0
- `accuracy` well need to look into, given the -1 values
- `priority`, which we can use as is
- `move_class`, which we can use as is (details about this bellow)
- `effect_id`, which we can use as is (details about this bellow)
- `effect_chance`, which we can use as is
  - For any `effect_chance == np.nan` values we will use 0

In [15]:
move_list

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
0,1,pound,1,normal,40.0,35,100.0,0,10,2,1,NaN,NaN,NaN
1,2,karate-chop,1,fighting,50.0,25,100.0,0,10,2,8,NaN,NaN,NaN
2,3,double-slap,1,normal,15.0,10,85.0,0,10,2,10,NaN,NaN,NaN
3,4,comet-punch,1,normal,18.0,15,85.0,0,10,2,10,NaN,NaN,NaN
4,5,mega-punch,1,normal,80.0,20,85.0,0,10,2,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,463,magma-storm,4,fire,100.0,5,75.0,0,10,3,24,NaN,NaN,7.0
463,464,dark-void,4,dark,NaN,10,80.0,0,11,1,13,NaN,NaN,5.0
464,465,seed-flare,4,grass,120.0,5,85.0,0,10,3,3,40.0,-2.0,4.0
465,466,ominous-wind,4,ghost,60.0,5,100.0,0,10,3,112,10.0,NaN,NaN


In [16]:
move_list.describe()

,id,generation_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
count,467.000000,467.000000,265.000000,467.000000,339.000000,467.000000,467.000000,467.000000,467.000000,94.000000,74.000000,119.000000
mean,234.000000,2.351178,74.418868,15.903640,93.601770,-0.008565,9.436831,1.862955,61.468951,30.851064,1.229730,3.403361
std,134.955548,1.192256,36.027058,8.833299,13.798126,0.907723,1.673376,0.757267,67.299317,28.118779,7.956297,1.684019
min,1.000000,1.000000,10.000000,1.000000,-1.000000,-7.000000,1.000000,1.000000,0.000000,10.000000,-2.000000,1.000000
25%,117.500000,1.000000,50.000000,10.000000,90.000000,0.000000,10.000000,1.000000,6.000000,10.000000,-1.000000,2.000000
50%,234.000000,2.000000,70.000000,15.000000,100.000000,0.000000,10.000000,2.000000,26.000000,20.000000,-1.000000,3.000000
75%,350.500000,3.000000,90.000000,20.000000,100.000000,0.000000,10.000000,2.000000,111.500000,30.000000,1.000000,5.000000
max,467.000000,4.000000,250.000000,40.000000,100.000000,5.000000,14.000000,3.000000,219.000000,100.000000,50.000000,7.000000


In [17]:
# Check missing values
move_list.isna().sum()

id                 0
identifier         0
generation_id      0
type_id            0
power            202
pp                 0
accuracy         128
priority           0
target_id          0
move_class         0
effect_id          0
effect_chance    373
effect_amt       393
effect_stat      348
dtype: int64

### Accuracy

In [18]:
move_list[move_list['accuracy'] < 0]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
11,12,guillotine,1,normal,NaN,5,-1.0,0,10,2,20,NaN,NaN,NaN
31,32,horn-drill,1,normal,NaN,5,-1.0,0,10,2,20,NaN,NaN,NaN
89,90,fissure,1,ground,NaN,5,-1.0,0,10,2,20,NaN,NaN,NaN
328,329,sheer-cold,3,ice,NaN,5,-1.0,0,10,3,20,NaN,NaN,NaN


### Special case moves

In [19]:
# sanity check that curse has ghost type
move_list[move_list['identifier'] == 'curse']

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
173,174,curse,2,ghost,NaN,10,NaN,0,1,1,77,NaN,NaN,NaN


### Power 

In [20]:
special_power_calc_moves = [
    'low-kick',
    'grass-knot',
    'return',
    'frustration',
    'flail',
    'reversal',
    'counter',
    'mirror-coat',
    'eruption',
    'gyro-ball',
]
move_list[move_list['identifier'].isin(special_power_calc_moves)]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
66,67,low-kick,1,fighting,NaN,20,100.0,0,10,2,35,NaN,NaN,NaN
67,68,counter,1,fighting,NaN,20,100.0,-5,1,2,36,NaN,NaN,NaN
174,175,flail,2,normal,NaN,15,100.0,0,10,2,78,NaN,NaN,NaN
178,179,reversal,2,fighting,NaN,15,100.0,0,10,2,78,NaN,NaN,NaN
215,216,return,2,normal,NaN,20,100.0,0,10,2,96,NaN,NaN,NaN
217,218,frustration,2,normal,NaN,20,100.0,0,10,2,98,NaN,NaN,NaN
242,243,mirror-coat,2,psychic,NaN,20,100.0,-5,1,3,110,NaN,NaN,NaN
283,284,eruption,3,fire,150.0,5,100.0,0,11,3,143,NaN,NaN,NaN
359,360,gyro-ball,4,steel,NaN,5,100.0,0,10,2,172,NaN,NaN,NaN
446,447,grass-knot,4,grass,NaN,20,100.0,0,10,3,35,NaN,NaN,NaN


Looking at the output above, I can see that moves that have a `power == np.nan` for moves that have special damage calculations have similair `effect_id`'s. For example: both Grass Knot and Low Kick deal more damage on heavier targets.

### PP

In [21]:
move_list[move_list['pp'] < 5]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
164,165,struggle,1,normal,50.0,1,NaN,0,8,2,68,NaN,NaN,NaN
165,166,sketch,2,normal,NaN,1,NaN,0,10,1,69,NaN,NaN,NaN


### Accuracy

In [38]:
sorted(move_list['accuracy'].unique())

[-1.0, 50.0, 55.0, 60.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, nan]

In [36]:
move_list[move_list['accuracy'] < 0]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
11,12,guillotine,1,normal,NaN,5,-1.0,0,10,2,20,NaN,NaN,NaN
31,32,horn-drill,1,normal,NaN,5,-1.0,0,10,2,20,NaN,NaN,NaN
89,90,fissure,1,ground,NaN,5,-1.0,0,10,2,20,NaN,NaN,NaN
328,329,sheer-cold,3,ice,NaN,5,-1.0,0,10,3,20,NaN,NaN,NaN


It seems that all moves with `accuracy < 0` are [OHKO moves](https://bulbapedia.bulbagarden.net/wiki/One-hit_knockout_move#Generation_III_onward). We can simply use these values as is, since all OHKO moves follow the same accuracy rules.

In [22]:
move_list[move_list['accuracy'].isna()]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
13,14,swords-dance,1,normal,NaN,20,NaN,0,7,1,16,NaN,2.0,1.0
17,18,whirlwind,1,normal,NaN,20,NaN,-6,10,1,0,NaN,NaN,NaN
45,46,roar,1,normal,NaN,20,NaN,-6,10,1,0,NaN,NaN,NaN
53,54,mist,1,ice,NaN,30,NaN,0,4,1,33,NaN,NaN,NaN
73,74,growth,1,normal,NaN,20,NaN,0,7,1,16,NaN,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,443,magnet-bomb,4,steel,60.0,20,NaN,0,10,2,1,NaN,NaN,NaN
445,446,stealth-rock,4,rock,NaN,20,NaN,0,6,1,214,NaN,NaN,NaN
454,455,defend-order,4,bug,NaN,10,NaN,0,7,1,158,NaN,NaN,NaN
455,456,heal-order,4,bug,NaN,10,NaN,0,7,1,46,NaN,NaN,NaN


It seems there are a few cases as to why accuracy would be `np.nan`:
- The move is a self buffing move (like Swords Dance) or stage altering (like Stealth Rock) which are unaffected by accuracy
- The move's accuracy is dependent on other factors (like with [Whirlwind](https://bulbapedia.bulbagarden.net/wiki/Whirlwind_(move)#Generations_III_and_IV) for example)
- The move bypasses any accuracy checks (like with [Aerial Ace](https://bulbapedia.bulbagarden.net/wiki/Aerial_Ace_(move)#Generation_III_onward) for example)
- It has hardcoded rules for targeting (like with [Snatch](https://bulbapedia.bulbagarden.net/wiki/Snatch_(move)#Generations_III_and_IV) for example)

### Target ID

In [24]:
move_list[move_list['target_id'] != 10]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
12,13,razor-wind,1,normal,80.0,10,100.0,0,11,3,21,NaN,NaN,NaN
13,14,swords-dance,1,normal,NaN,20,NaN,0,7,1,16,NaN,2.0,1.0
36,37,thrash,1,normal,120.0,10,100.0,0,8,2,28,NaN,NaN,NaN
38,39,tail-whip,1,normal,NaN,30,100.0,0,11,1,17,NaN,-1.0,2.0
42,43,leer,1,normal,NaN,30,100.0,0,11,1,17,NaN,-1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,446,stealth-rock,4,rock,NaN,20,NaN,0,6,1,214,NaN,NaN,NaN
454,455,defend-order,4,bug,NaN,10,NaN,0,7,1,158,NaN,NaN,NaN
455,456,heal-order,4,bug,NaN,10,NaN,0,7,1,46,NaN,NaN,NaN
460,461,lunar-dance,4,psychic,NaN,10,NaN,0,7,1,218,NaN,NaN,NaN


After some research, I found that target id dictates what the move targets. So for stat boosting on self target id = 7, stealth rock and other entry hazards target id = 6, etc. This is not a very useful feature for the agent to learn, so we will not include it in the observation space.

### Move class

In [26]:
print(move_list['move_class'].unique())
move_category_labels = {
    1: 'status',
    2: 'physical',
    3: 'special'
}
print(move_category_labels)

[2 3 1]
{1: 'status', 2: 'physical', 3: 'special'}


I tought I would need to encode move categories as well, but the simulator already has those represented in numerical values.

### Effect ID

In [ ]:
move_list[move_list['effect_id'] == 1]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
0,1,pound,1,normal,40.0,35,100.0,0,10,2,1,NaN,NaN,NaN
4,5,mega-punch,1,normal,80.0,20,85.0,0,10,2,1,NaN,NaN,NaN
9,10,scratch,1,normal,40.0,35,100.0,0,10,2,1,NaN,NaN,NaN
10,11,vice-grip,1,normal,55.0,30,100.0,0,10,2,1,NaN,NaN,NaN
14,15,cut,1,normal,50.0,30,95.0,0,10,2,1,NaN,NaN,NaN
16,17,wing-attack,1,flying,60.0,35,100.0,0,10,2,1,NaN,NaN,NaN
20,21,slam,1,normal,80.0,20,75.0,0,10,2,1,NaN,NaN,NaN
21,22,vine-whip,1,grass,45.0,25,100.0,0,10,2,1,NaN,NaN,NaN
24,25,mega-kick,1,normal,120.0,5,75.0,0,10,2,1,NaN,NaN,NaN
29,30,horn-attack,1,normal,65.0,25,100.0,0,10,2,1,NaN,NaN,NaN


### Effect Change

In [39]:
sorted(move_list['effect_chance'].unique())

[nan, 10.0, 20.0, 30.0, 40.0, 50.0, 70.0, 100.0]

In [27]:
move_list[move_list['effect_chance'].isna()]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
0,1,pound,1,normal,40.0,35,100.0,0,10,2,1,NaN,NaN,NaN
1,2,karate-chop,1,fighting,50.0,25,100.0,0,10,2,8,NaN,NaN,NaN
2,3,double-slap,1,normal,15.0,10,85.0,0,10,2,10,NaN,NaN,NaN
3,4,comet-punch,1,normal,18.0,15,85.0,0,10,2,10,NaN,NaN,NaN
4,5,mega-punch,1,normal,80.0,20,85.0,0,10,2,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,461,lunar-dance,4,psychic,NaN,10,NaN,0,7,1,218,NaN,NaN,NaN
461,462,crush-grip,4,normal,NaN,5,100.0,0,10,2,190,NaN,NaN,NaN
462,463,magma-storm,4,fire,100.0,5,75.0,0,10,3,24,NaN,NaN,7.0
463,464,dark-void,4,dark,NaN,10,80.0,0,11,1,13,NaN,NaN,5.0


### Effect Amount

In [29]:
list(move_list[move_list['effect_amt'].notna()]['identifier'])

['swords-dance',
 'sand-attack',
 'tail-whip',
 'leer',
 'growl',
 'sonic-boom',
 'acid',
 'bubble-beam',
 'aurora-beam',
 'growth',
 'string-shot',
 'dragon-rage',
 'psychic',
 'meditate',
 'agility',
 'screech',
 'double-team',
 'harden',
 'smokescreen',
 'withdraw',
 'barrier',
 'constrict',
 'amnesia',
 'kinesis',
 'bubble',
 'flash',
 'acid-armor',
 'sharpen',
 'flame-wheel',
 'cotton-spore',
 'scary-face',
 'mud-slap',
 'octazooka',
 'icy-wind',
 'charm',
 'steel-wing',
 'sacred-fire',
 'sweet-scent',
 'iron-tail',
 'metal-claw',
 'crunch',
 'shadow-ball',
 'rock-smash',
 'tail-glow',
 'luster-purge',
 'mist-ball',
 'feather-dance',
 'crush-claw',
 'meteor-mash',
 'fake-tears',
 'overheat',
 'rock-tomb',
 'metal-sound',
 'muddy-water',
 'iron-defense',
 'howl',
 'mud-shot',
 'psycho-boost',
 'hammer-arm',
 'rock-polish',
 'night-slash',
 'bug-buzz',
 'focus-blast',
 'energy-ball',
 'earth-power',
 'nasty-plot',
 'mud-bomb',
 'psycho-cut',
 'mirror-shot',
 'flash-cannon',
 'draco-

All the moves in the output above have either the primary or secondary effect of altering pokemons stats by some amount. The `effect_amt` column shows the amount by which the stat is altered. This is a very useful feature for the agent to learn, so we will include it in the observation space.

In [30]:
move_list['effect_stat'].unique()

array([nan,  1.,  2.,  3.,  6.,  4.,  5.,  7.])

In [31]:
move_list[move_list['effect_stat'].notna()]

,id,identifier,generation_id,type_id,power,pp,accuracy,priority,target_id,move_class,effect_id,effect_chance,effect_amt,effect_stat
6,7,fire-punch,1,fire,75.0,15,100.0,0,10,2,5,10.0,NaN,1.0
7,8,ice-punch,1,ice,75.0,15,100.0,0,10,2,5,10.0,NaN,2.0
8,9,thunder-punch,1,electric,75.0,15,100.0,0,10,2,5,10.0,NaN,3.0
13,14,swords-dance,1,normal,NaN,20,NaN,0,7,1,16,NaN,2.0,1.0
19,20,bind,1,normal,15.0,20,85.0,0,10,2,24,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,441,gunk-shot,4,poison,120.0,5,80.0,0,10,2,5,30.0,NaN,4.0
450,451,charge-beam,4,electric,50.0,10,90.0,0,10,3,2,70.0,1.0,3.0
462,463,magma-storm,4,fire,100.0,5,75.0,0,10,3,24,NaN,NaN,7.0
463,464,dark-void,4,dark,NaN,10,80.0,0,11,1,13,NaN,NaN,5.0


I could not find out what `effect_stat` is for, so I will not include it in the observation space.

## Non-volatile status effects (like sleep, poison, etc)

... TODO FIX THIS SHIT

## Held item

... TODO FIX THIS SHIT

## Weight, friendship and gender

| Name       | Min | Max   | Type     |
|------------|-----|-------|----------|
| weight     | 1   | 9500  | Discrete |
| friendship | 0   | 254   | Discrete |
| gender     | 0   | 2     | Discrete |

These are not really that special. Every pokemon has a weight defined in the data files. Friendship is just a integer value that can be between 0 and 254 (inclusive). And their are 3 possible values for gender: male, female or genderless, which need to be label encoded.

In [32]:
pokemon_stats.describe()

,ndex,hp,attack,defense,sp. atk,sp. def,speed,height,weight,base exp.,gen
count,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000
mean,247.000000,67.730223,73.496957,70.109533,67.981744,69.158215,65.440162,11.845842,590.900609,145.955375,2.401623
std,142.461106,27.580375,29.168464,30.703012,28.515038,27.884112,27.223685,11.344592,960.959391,81.698347,1.135602
min,1.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,2.000000,1.000000,36.000000,1.000000
25%,124.000000,50.000000,50.000000,50.000000,45.000000,50.000000,45.000000,6.000000,99.000000,66.000000,1.000000
50%,247.000000,65.000000,72.000000,65.000000,65.000000,65.000000,65.000000,10.000000,295.000000,147.000000,2.000000
75%,370.000000,80.000000,90.000000,85.000000,90.000000,85.000000,85.000000,15.000000,608.000000,178.000000,3.000000
max,493.000000,255.000000,165.000000,230.000000,154.000000,230.000000,160.000000,145.000000,9500.000000,635.000000,4.000000


In [33]:
pokemon_stats.isna().sum()

ndex           0
name           0
type 1         0
type 2       270
hp             0
attack         0
defense        0
sp. atk        0
sp. def        0
speed          0
height         0
weight         0
base exp.      0
gen            0
dtype: int64

In [34]:
pb.conf.global_settings.POSSIBLE_GENDERS

['male', 'female', 'genderless']

Pokemon are either:
- male or female
- always genderless

In [35]:
gender_encoder = LabelEncoder()
gender_encoder.fit(pb.conf.global_settings.POSSIBLE_GENDERS)

def get_random_gender_mf():
    return gender_encoder.transform(random.choice(['male', 'female']))

def get_gender_encoding(gender: str):
    return gender_encoder.transform([gender])[0]

def get_gender_decoding(gender: int):
    return gender_encoder.inverse_transform([gender])[0]

# for c in gender_encoder.classes_:
#     print(f'{c} -> {get_gender_encoding(c)} -> {get_gender_decoding(get_gender_encoding(c))}')

## Lingering effects on the battlefield

... TODO FIX THIS SHIT

## Volatile statusus effects

... TODO FIX THIS SHIT

## Incorperation into environment

TODO incorperate this into environment

## Conclusion

### On removing stuff from the observation space

It might be interesting to see if we remove some stuff from the observation space, what it would do to the model. For example: would it be able to learn on its own that swords dance increases attack? Or would it be able to learn that a move is a physical move by looking at the move's power + the pokemons attack stat + its own defense stat? This would be an interesting experiment to run.